In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

import plotly.graph_objects as go

In [2]:
# import survey data

data = pd.read_excel('/Users/t0l0bkk/Documents/TTL/github/survey-tool/data/raw_data.xlsx',sheet_name = 'sample')

In [3]:
# define functions to prepare data for visualization
class metrics_prep():
    
    def __init__(self, data):
        
        # outputs
        self.df = data
        self.m1 = self.get_m1()
        self.m2 = self.get_m2()
        self.m3 = self.get_m3()
        self.m4 = self.get_m4()
        
    def get_m1(self):
        
        m1_total_num_issues = self.df['Issues'].count()
        
        return m1_total_num_issues

    def get_m2(self):
        
        self.df['Easy_to_Fix'] = ['Yes' if p + h + v + n <= 2 else 'No' for p, h, v, n in zip(self.df['Physical'], self.df['Hearing'], self.df['Vision'], self.df['Neurodiverse'])]
        m2_easy_to_fix = self.df.groupby(['Easy_to_Fix'])['Issues'].count().reset_index(drop = False)
        m2_easy_to_fix['Percentage'] = m2_easy_to_fix['Issues'] / m2_easy_to_fix['Issues'].sum()

        return m2_easy_to_fix
    
    def get_m3(self):
        
        m3_types_of_disabilities_affected = self.df[['Physical', 'Hearing', 'Vision', 'Neurodiverse']].sum().reset_index(drop = False)
        m3_types_of_disabilities_affected.columns = ['Types_of_Disabilities_Affected', 'Issues']

        return m3_types_of_disabilities_affected
    
    def get_m4(self):
        
        m4_issues_per_l2_cat = self.df.groupby(['L2'])['Issues'].count().reset_index(drop = False)
        
        return m4_issues_per_l2_cat

In [4]:
# get metric results
m = metrics_prep(data)
m1 = m.m1
m2 = m.m2
m3 = m.m3
m4 = m.m4

In [5]:
fig_m1 = go.Figure()

# Add card for Total Issues
fig_m1.add_trace(go.Indicator(
    mode = "number",
    value = m1,
    title = {"text": "Total Number of Issues", "font": {"size": 24}},
    domain={'x': [0.25, 0.75], 'y': [0, 1]},  # Left card
    number = {"font": {"size": 36}},
))

fig_m1.update_layout(width=500, height=400 )
fig_m1.show()

In [6]:
fig_m2 = go.Figure()

fig_m2 = px.bar(
    m2,
    x='Easy_to_Fix',
    y='Percentage',
    text=[f"{round(p * 100, 1)}%" for p in m2['Percentage']],
    labels={'Easy_to_Fix': 'Easy to Fix', 'Issues': 'Number of Issues'},
    title='Issues by Easy to Fix Status'
)

# Update layout for better appearance
fig_m2.update_traces(textposition='outside')  # Position text labels outside of bars
fig_m2.update_layout(xaxis_title='Easy to Fix'
                     , template='plotly_white'
                     , width=500  # Set the width of the figure
                     , height=400  # Optional: set height for better proportions
    )

fig_m2.update_traces(width=0.4) 
fig_m2.update_yaxes(range=[0, 1], tickformat=".0%") 

fig_m2.show()

In [7]:
m3

,Types_of_Disabilities_Affected,Issues
0,Physical,71
1,Hearing,38
2,Vision,34
3,Neurodiverse,19


In [8]:
# Create fig_m3 (Horizontal Bar Chart)
fig_m3 = px.bar(
    m3,
    x='Issues',
    y='Types_of_Disabilities_Affected',
    orientation='h',  # Horizontal bar chart
    text='Issues',  # Show issue numbers as text labels
    color='Types_of_Disabilities_Affected',  # Color by type
    labels={'Types_of_Disabilities_Affected': 'Types of Disabilities Affected', 'Issues': 'Number of Issues'},
    title='Number of Issues by Types of Disabilities Affected'
)

# Update layout for fig_m3
fig_m3.update_traces(textposition='outside')  # Position text labels outside of bars
fig_m3.update_layout(
    xaxis_title='Number of Issues',
    template='plotly_white',
    yaxis=dict(ticklabelposition="inside", automargin=True),  # Use automargin to create space
    # width=500,  # Set the width of the figure
    # height=400  # Optional: set height for better proportions
)

# Show the figure
fig_m3.show()

In [33]:
import dash
from dash import dcc, html
import plotly.graph_objects as go

app = dash.Dash(__name__)

# Use the image from the assets folder
floorplan_image = 'https://www.lcpmedia.com/hs-fs/hubfs/C1.jpg?width=3001&name=C1.jpg'  # Adjusted path

# Actual dimensions of the floorplan image
image_width = 3000  # Image width in pixels
image_height = 2000  # Image height in pixels

# Scale factor
scale_factor = 0.3

# New dimensions of the floorplan image
scaled_image_width = image_width * scale_factor
scaled_image_height = image_height * scale_factor

# Sample data for pins (adjusted to pixel coordinates)
pins = {
    'Issue A': {'x': 600 * scale_factor, 'y': 1200 * scale_factor, 'issue': 'Leaking Faucet'},  # Coordinates in pixels
    'Issue B': {'x': 1500 * scale_factor, 'y': 600 * scale_factor, 'issue': 'Broken Window'},
    'Issue C': {'x': 1800 * scale_factor, 'y': 1000 * scale_factor, 'issue': 'No Power'},
    'Issue D': {'x': 900 * scale_factor, 'y': 800 * scale_factor, 'issue': 'Clogged Sink'},
}

# Create figure
fig_map = go.Figure()

# Add floorplan image
fig_map.add_layout_image(
    dict(
        source=floorplan_image,
        x=0,
        y=scaled_image_height,  # Adjusted to place the image correctly
        xref="x",
        yref="y",
        sizex=scaled_image_width,
        sizey=scaled_image_height,
        opacity=1,
        layer="below"
    )
)

# Add pins to the floorplan
for room, data in pins.items():
    fig_map.add_trace(go.Scatter(
        x=[data['x']],
        y=[data['y']],
        mode='markers+text',
        marker=dict(size=15, color='red'),
        text=room,
        textposition='top center',
        hoverinfo='text',
        name=room,
        customdata=[data['issue']],  # Wrap in a list
    ))

# Update layout
fig_map.update_layout(
    title="Indoor Floorplan",
    xaxis=dict(showgrid=False, zeroline=False, range=[0, scaled_image_width]),  # Set x-axis range
    yaxis=dict(showgrid=False, zeroline=False, range=[0, scaled_image_height]),  # Set y-axis range
    xaxis_title="",
    yaxis_title="",
    showlegend=False,
    width=scaled_image_width,  # Set width to match image
    height=scaled_image_height,  # Set height to match image
)

# Show the figure
fig_map.show()


In [34]:
# Initialize the Dash app
app = dash.Dash(__name__)

# App Layout
app.layout = html.Div(children=[
    
    html.H1(children='Survey Results Dashboard'),

    html.Div(id = 'first_row'
        , children=[
            dcc.Graph(
                id='total-issues-graph',
                figure=fig_m1,
                style={'flex': '1'}  # Adjust flex to take up equal space
            ),
            dcc.Graph(
                id='issues-by-easy-to-fix-graph',
                figure=fig_m2,
                style={'flex': '1'}  # Adjust flex to take up equal space
            ),
            dcc.Graph(
                id='types-of-disabilities-affected',
                figure=fig_m3,
                style={'flex': '2'}  # Adjust flex to take up equal space
            )
        ],
        style={'display': 'flex', 'flex-direction': 'row'}  # Flexbox for horizontal alignment
    ),

    # # Placeholder for future graphs in subsequent rows
    html.Div(id='second-row'
        , children=[
            dcc.Graph(
                id='floor-plan',
                figure = fig_map,
                style={'flex': '4'} 
            )
        ],
        style={'display': 'flex', 'flex-direction': 'row'} 
    )
])


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
